In [1]:
import pandas as pd
import json, os

In [5]:
invoiceNo = '869706'
BY_INVOICE_ADDRESS = os.getenv('BY_INVOICE_ADDRESS')

In [6]:
df = pd.read_html(BY_INVOICE_ADDRESS+invoiceNo)

In [7]:
itemList = df[9].drop(columns=[4,5], axis=1)

In [8]:
itemList.columns = itemList.iloc[0]
itemList = itemList[1:].reset_index(drop=True)
itemList['DESCRIPTION'] = itemList['DESCRIPTION'].str.split(',')

In [9]:
new_list = pd.DataFrame(columns=['Item', 'QTY', 'Color', 'Price'])

# Expand items list
for i in range(len(itemList)):
    for des in itemList['DESCRIPTION'][i]:
        des = des.strip()
        if des == '#1' or des == '#2':
            continue
        else:
            color, qty = des.split(':')
            new_list.loc[len(new_list)] = [itemList['ITEM'][i], qty, color.strip('#'), itemList['UNIT PRICE'][i]]

new_list['QTY'] = new_list['QTY'].str.replace('\(.\)', '', regex=True)
new_list.drop(new_list[new_list['QTY']=='0'].index, inplace=True)

In [10]:
# Load BY inventory list

BY_list = pd.read_excel('test_data\BY_InventoryListAll.xls', skiprows=3)

# Preprocessing
BY_list = BY_list[['Item Name', 'Color', 'Barcode']]
BY_list['Item'] = BY_list['Item Name'].str.replace(' ', '')
BY_list['Item'] = BY_list['Item'].str.replace('-', '')
BY_list['Item'] = BY_list['Item'].str.replace('TOP', 'TP')
BY_list['Item'] = BY_list['Item'].str.replace('TPSDM', 'TOPSDM')

WARNING *** file size (5316480) not 512 + multiple of sector size (512)


In [11]:
new_list = new_list.merge(BY_list, how='left', left_on=['Item', 'Color'], right_on=['Item', 'Color'])

In [ ]:
try:
    os.mkdir('./'+invoiceNo)
except:
    print(f'{invoiceNo} directory already exist!')

df_receiving = new_list[['Barcode', 'QTY', 'Price']]
df_receiving.columns=['Code', 'Qty on Ord', 'Cost']

df_receiving.to_csv(f'{invoiceNo}/{invoiceNo}_receiving.txt', sep='\t',index=False)

In [ ]:
df_new_item = new_list[['Barcode']]
df_new_item.insert(loc=len(df_new_item.columns),column='Description', value=new_list['Item Name'].str.cat(' '+new_list['Color']))
df_new_item.insert(loc=len(df_new_item.columns), column='Ext Desc', value=new_list['Item Name'].str.cat(' #'+new_list['Color']))
df_new_item.insert(loc=len(df_new_item.columns), column='Cost', value=new_list['Price'].str.strip('$ '))
df_new_item.insert(loc=len(df_new_item.columns), column='Regular Price', value=0)
df_new_item.insert(loc=len(df_new_item.columns), column='Department', value=pd.NA)

df_new_item.to_csv(f'{invoiceNo}/{invoiceNo}_new_item.txt', sep='\t', index=False)

In [ ]:
df_supplier = new_list[['Barcode']]
df_supplier.insert(loc=len(df_supplier.columns),column='Cost', value=new_list['Price'])
df_supplier.insert(loc=len(df_supplier.columns),column='ReorderNo', value=new_list['Item Name'])
df_supplier.insert(loc=len(df_supplier.columns),column='MinimumOrder', value=0)
df_supplier.insert(loc=len(df_supplier.columns),column='MPQ', value=0)

df_supplier.to_csv(f'{invoiceNo}/{invoiceNo}_supplier.txt', sep='\t', index=False)

In [ ]:
new_list